In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import os 
from PIL import Image,ImageOps
import PIL
import random

c:\users\19706\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\19706\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\19706\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\19706\appd

fetch Data 

In [2]:
path = os.getcwd()
path += '\\101_ObjectCategories'
contents = os.listdir(path)
name_tags = {k: v for v, k in enumerate(contents)} # conver to dict of names:indices
reverse_of_name_tags = {v: k for v, k in enumerate(contents)}# reverse dictionary for future use\
Data = []
image_shape = (512 , 512)
dropoutVec = [1.0] * 8 + [0.7] * 2 + [0.5] * 2 + [0.5] * 1 + [0.5, 1.] * 5
learningRate=0.002
beta1 = 0.9
beta2 = 0.999
batchSize = 100
epochs = 10

In [3]:
baseheight = 560

for folder in contents: # read Data 
    folder_contents=os.listdir(path+"\\"+folder)
    for file in folder_contents:
        img = Image.open(path+"\\"+folder+"\\"+file)
        img = ImageOps.fit(img, image_shape, Image.ANTIALIAS)
        image= np.array(img)
        #print(image.shape)
        #plt.imshow(image)
        if image.shape == (512,512 ,3): # remove some irregular images
            Data.append([image , name_tags[folder]]) 

In [4]:
total_files = len(Data)

In [5]:
# seperate data 
X = [x[0] for x in Data]
Y = [y[1] for y in Data]

In [6]:
indices = np.array(range(total_files))
random.shuffle(indices)
train_indices = indices[:int(0.8*total_files)]
test_indices = indices[int(0.8*total_files):]

In [7]:
Xtrain,Ytrain = np.array([(X[i]) for i in train_indices]),np.array([Y[i] for i in train_indices]) 
Xtest,Ytest = np.array([X[i] for i in test_indices]),np.array([Y[i] for i in test_indices]) 

In [8]:
Ytrain[152]

19

In [9]:
def genData(inputs=Xtrain, targets = Ytrain , batchSize = 200):
    batchX = np.zeros((batchSize, image_shape[0],image_shape[1],3 ))    
    batchY = np.zeros((batchSize, 1))
    idx = 0       
    while True: # to make sure we never reach the end
        counter = 0
        while counter<=batchSize-1:
            dataIdx = np.random.randint(batchSize-1) 
            batchX[counter] = inputs[dataIdx]
            batchY[counter] = targets[dataIdx]
            counter += 1
        yield (batchX, batchY)

In [12]:
def weight_ones(shape, name):
    initial = tf.constant(1.0, shape=shape, name=name)
    return tf.Variable(initial)

def weight_xavi_init(shape, name):
    initial = tf.get_variable(name=name, shape=shape, initializer=tf.contrib.layers.xavier_initializer())
    return initial


def bias_variable(shape, name):
    initial = tf.constant(0.1, shape=shape, name=name)
    return tf.Variable(initial)


class Network(object):

    def __init__(self, dropout, image_shape):
        """ We put a few counters to see how many times we called each function """
        self._dropout_vec = dropout
        self._image_shape = image_shape
        self._count_conv = 0
        self._count_pool = 0
        self._count_bn = 0
        self._count_activations = 0
        self._count_dropouts = 0
        self._count_fc = 0
        self._count_lstm = 0
        self._count_soft_max = 0
        self._conv_kernels = []
        self._conv_strides = []
        self._weights = {}
        self._features = {}

    """ Our conv is currently using bias """

    def conv(self, x, kernel_size, stride, output_size, padding_in='SAME'):
        self._count_conv += 1

        filters_in = x.get_shape()[-1]
        shape = [kernel_size, kernel_size, filters_in, output_size]

        weights = weight_xavi_init(shape, 'W_c_' + str(self._count_conv))
        bias = bias_variable([output_size], name='B_c_' + str(self._count_conv))

        self._weights['W_conv' + str(self._count_conv)] = weights
        self._conv_kernels.append(kernel_size)
        self._conv_strides.append(stride)

        conv_res = tf.add(tf.nn.conv2d(x, weights, [1, stride, stride, 1], padding=padding_in,
                                       name='conv2d_' + str(self._count_conv)), bias,
                          name='add_' + str(self._count_conv))

        self._features['conv_block' + str(self._count_conv - 1)] = conv_res

        return conv_res

    def max_pool(self, x, ksize=3, stride=2):
        self._count_pool += 1
        return tf.nn.max_pool(x, ksize=[1, ksize, ksize, 1], strides=[1, stride, stride, 1],
                              padding='SAME', name='max_pool' + str(self._count_pool))

    def bn(self, x):
        self._count_bn += 1
        return tf.contrib.layers.batch_norm(x, is_training=False,
                                            updates_collections=None, scope='bn' + str(self._count_bn))

    def activation(self, x):
        self._count_activations += 1
        return tf.nn.relu(x, name='relu' + str(self._count_activations))

    def dropout(self, x):
        print ("Dropout", self._count_dropouts)
        self._count_dropouts += 1
        output = tf.nn.dropout(x, self._dropout_vec[self._count_dropouts - 1],
                               name='dropout' + str(self._count_dropouts))

        return output

    def fc(self, x, output_size):
        self._count_fc += 1
        filters_in = x.get_shape()[-1]
        shape = [filters_in, output_size]

        weights = weight_xavi_init(shape, 'W_f_' + str(self._count_fc))
        bias = bias_variable([output_size], name='B_f_' + str(self._count_fc))

        return tf.nn.xw_plus_b(x, weights, bias, name='fc_' + str(self._count_fc))

    def conv_block(self, x, kernel_size, stride, output_size, padding_in='SAME'):
        print( " === Conv", self._count_conv, "  :  ", kernel_size, stride, output_size)
        with tf.name_scope("conv_block" + str(self._count_conv)):
            x = self.conv(x, kernel_size, stride, output_size, padding_in=padding_in)
            x = self.bn(x)
            x = self.dropout(x)

            return self.activation(x)

    def fc_block(self, x, output_size):
        print (" === FC", self._count_fc, "  :  ", output_size)
        with tf.name_scope("fc" + str(self._count_fc + 1)):
            x = self.fc(x, output_size)
            x = self.dropout(x)
            self._features['fc_block' + str(self._count_fc + 1)] = x
            return self.activation(x)

    def get_weigths_dict(self):
        return self._weights

    def get_feat_tensors_dict(self):
        return self._features
def load_imitation_learning_network(input_image, dropout):
    x = input_image
    network_manager = Network(dropout, tf.shape(x))
    with tf.name_scope("ConvNet"):
        """conv1"""  # kernel sz, stride, num feature maps
        xc = network_manager.conv_block(x, 5, 2, 32, padding_in='VALID')
        print( xc)
        xc = network_manager.conv_block(xc, 3, 1, 32, padding_in='VALID')
        print(xc)

        """conv2"""
        xc = network_manager.conv_block(xc, 3, 2, 64, padding_in='VALID')
        print(xc)
        xc = network_manager.conv_block(xc, 3, 1, 64, padding_in='VALID')
        print(xc)

        """conv3"""
        xc = network_manager.conv_block(xc, 3, 2, 128, padding_in='VALID')
        print(xc)
        xc = network_manager.conv_block(xc, 3, 1, 128, padding_in='VALID')
        print(xc)

        """conv4"""
        xc = network_manager.conv_block(xc, 3, 1, 256, padding_in='VALID')
        print(xc)
        xc = network_manager.conv_block(xc, 3, 1, 256, padding_in='VALID')
        print(xc)
        """mp3 (default values)"""

        """ reshape """
        x = tf.reshape(xc, [-1, int(np.prod(xc.get_shape()[1:]))], name='reshape')
        print (x)

        """ fc1 """
        x = network_manager.fc_block(x, 512)
        print (x)
        """ fc2 """
        x = network_manager.fc_block(x, 512)
    return x

In [13]:
def controlNet(inputs, targets, shape, dropoutVec, scopeName = 'controlNET'):
    """
        Get one image/sequence of images to predict control operations for controling the vehicle
        inputs: N Batch of M images in order
        shape: [BatchSize, SeqSize, FrameHeight, FrameWeight, Channels]
        phase: placeholder for training
        scopeName: TensorFlow Scope Name to separate nets in the graph
    """
    with tf.variable_scope(scopeName) as scope:
        with tf.name_scope("Network"):           
            networkTensor = load_imitation_learning_network(inputs[0], dropoutVec)
            solverList = []
            lossList = []
            trainVars = tf.trainable_variables()
            with tf.name_scope("ConvNet"):             
                contLoss = tf.reduce_mean(tf.square(tf.subtract(networkTensor, targets[0]))) #+ tf.add_n([tf.nn.l2_loss(v) for v in trainVars]) * L2NormConst
                contSolver = tf.train.AdamOptimizer(learning_rate=learningRate, beta1=beta1,
                                                    beta2=beta2).minimize(contLoss)
                solverList.append(contSolver)
                lossList.append(contLoss)  
                tf.summary.scalar("ConVnet_output", contLoss)        
        tensors = {
            'optimizers' : solverList,
            'losses' : lossList,
            'output' : networkTensor
        }
    return tensors

In [14]:
def Net( prefSize=(image_shape[0], image_shape[1], 3)):   
    shapeInput = [None, prefSize[0], prefSize[1], prefSize[2]]
    inputImages = tf.placeholder("float", shape=[None, prefSize[0], prefSize[1],
                                                                 prefSize[2]], name="input_image")      
    inputs = [inputImages]
    dout = tf.placeholder("float", shape=len(dropoutVec)) 
    targettag = tf.placeholder(tf.float32, shape=[None, 1], name="target_speed")   
    targets = [targettag]
    print('Building ControlNet ...')    
    controlOpTensors = controlNet(inputs, targets, shapeInput, dout,scopeName = 'controlNET')   
    tensors = {
            'inputs' : inputs,
            'targets' : targets,
            'dropoutVec' : dout,
            'output' : controlOpTensors
    }
    return tensors # [ inputs['inputImages','inputData'], targets['targetSpeed', 'targetController'],  'params', dropoutVec', output[optimizers, losses, branchesOutputs] ]

In [15]:
tf.reset_default_graph()
sessGraph = tf.Graph()
# GPU configuration
memory_fraction = 0.25
config = tf.ConfigProto(allow_soft_placement = True)
config.gpu_options.visible_device_list = '0'
config.gpu_options.per_process_gpu_memory_fraction = memory_fraction
tf.reset_default_graph()
sessGraph = tf.Graph()
# Prepare data generators
batchListGenTrain = []
#batchListGenVal = []
#batchListName = []

with tf.name_scope("ConvNet"):
    miniBatchGen = genData(Xtrain,Ytrain ,batchSize = batchSize)
    batchListGenTrain.append(miniBatchGen)
    #miniBatchGen = genData(Xval,Yval ,batchSize = batchSize)
    #batchListGenVal.append(miniBatchGen)
      
with sessGraph.as_default():
    sess = tf.Session(graph=sessGraph, config = config)
    with sess.as_default():        
        # build model
        print('Building Net ...')
        netTensors = Net(prefSize=(image_shape[0], image_shape[1], 3))
        print(netTensors['output'])
        print('Initialize Variables in the Graph ...')
        sess.run(tf.global_variables_initializer()) # initialize variables       
        # merge all summaries into a single op
        merged_summary_op = tf.summary.merge_all()
        saver = tf.train.Saver(write_version = saver_pb2.SaverDef.V2)
        if not(trainScratch):
            saver.restore(sess, "test/model.ckpt") # restore trained parameters
        # op to write logs to Tensorboard
        logsPath = './logs'
        modelPath = './test/'
        summary_writer = tf.summary.FileWriter(logsPath, graph=sessGraph)
        print('Start Training process ...')        
        steps = 0
        for epoch in range(epochs):
            tStartEpoch = time.time()
            print("Epoch:", epoch)
            iterations = Xtrain.shape[0]
            for j in range(iterations):
                steps += 1                
                xs , ys = next(batchListGenTrain)                   
                # augment images                    
                contSolver = netTensors['output']['optimizers'][i]#solverList[i]
                contLoss = netTensors['output']['losses'][i]#lossList[i]                                           
                # [ inputs['inputImages','inputData'], targets['targetSpeed', 'targetController'],  'params', dropoutVec', output[optimizers, losses, branchesOutputs] ]
                feedDict = {netTensors['inputs'][0]: xs,  netTensors['dropoutVec']: dropoutVec, 
                            netTensors['targets']: ys.reshape([batchSize,1]), 
                            }                       
                _,loss_value = sess.run([contSolver, contLoss], feed_dict = feedDict)                        
                # write logs at every iteration
                #feedDict = {netTensors['inputs'][0]: xs, netTensors['inputs'][1][0]: inputData[0], netTensors['inputs'][1][1]: inputData[1], netTensors['dropoutVec']: [1] * len(dropoutVec), netTensors['targets'][0]: ys[:,10].reshape([batchSize,1]), netTensors['targets'][1]: ys[:,0:3]}                       
                summary = merged_summary_op.eval(feed_dict=feedDict)
                summary_writer.add_summary(summary, epoch * iterations + j)
                print("  Train::: Epoch: %d, Step: %d, TotalSteps: %d, Loss: %g" % (epoch, epoch * batchSize + j, steps, loss_value), cBranchesOutList[i])                        
                if steps % 10 == 0: # clear_log
                    clear_output(wait=True)
                    
                if steps % 50 == 0 and steps!=0: # batchSize
                    print(j%50, '  Save Checkpoint ...')
                    if not os.path.exists(modelPath):
                        os.makedirs(modelPath)
                    checkpoint_path = os.path.join(modelPath, "model.ckpt")
                    filename = saver.save(sess, checkpoint_path)
                    print("Model saved in file: %s" % filename)                
                    # update new Losses and Optimizers 
                    print('Initialize Variables in the Graph ...')
                    # merge all summaries into a single op
                    merged_summary_op = tf.summary.merge_all()
                    sess.run(tf.global_variables_initializer())
                    saver.restore(sess, "test/model.ckpt") # restore trained parameters

                if steps % total_steps == 0 and steps!=0:
                    # finish the training
                    break
            if steps % total_steps == 0 and steps!=0:
                # finish the training
                print('Finalize the training and Save Checkpoint ...')
                if not os.path.exists(modelPath):
                    os.makedirs(modelPath)
                checkpoint_path = os.path.join(modelPath, "model.ckpt")
                filename = saver.save(sess, checkpoint_path)
                print("  Model saved in file: %s" % filename)
                break

            tStopEpoch = time.time()
            print( "  Epoch Time Cost:", round(tStopEpoch - tStartEpoch,2), "s")

Building Net ...
Building ControlNet ...
 === Conv 0   :   5 2 32
Dropout 0
Tensor("controlNET/Network/ConvNet/conv_block0/relu1:0", shape=(?, 254, 254, 32), dtype=float32)
 === Conv 1   :   3 1 32
Dropout 1
Tensor("controlNET/Network/ConvNet/conv_block1/relu2:0", shape=(?, 252, 252, 32), dtype=float32)
 === Conv 2   :   3 2 64
Dropout 2
Tensor("controlNET/Network/ConvNet/conv_block2/relu3:0", shape=(?, 125, 125, 64), dtype=float32)
 === Conv 3   :   3 1 64
Dropout 3
Tensor("controlNET/Network/ConvNet/conv_block3/relu4:0", shape=(?, 123, 123, 64), dtype=float32)
 === Conv 4   :   3 2 128
Dropout 4
Tensor("controlNET/Network/ConvNet/conv_block4/relu5:0", shape=(?, 61, 61, 128), dtype=float32)
 === Conv 5   :   3 1 128
Dropout 5
Tensor("controlNET/Network/ConvNet/conv_block5/relu6:0", shape=(?, 59, 59, 128), dtype=float32)
 === Conv 6   :   3 1 256
Dropout 6
Tensor("controlNET/Network/ConvNet/conv_block6/relu7:0", shape=(?, 57, 57, 256), dtype=float32)
 === Conv 7   :   3 1 256
Dropout 7

ResourceExhaustedError: OOM when allocating tensor with shape[774400,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: controlNET/W_f_1/Initializer/random_uniform/RandomUniform = RandomUniform[T=DT_INT32, _class=["loc:@controlNET/W_f_1/Assign"], dtype=DT_FLOAT, seed=0, seed2=0, _device="/job:localhost/replica:0/task:0/device:GPU:0"](controlNET/W_f_1/Initializer/random_uniform/shape)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'controlNET/W_f_1/Initializer/random_uniform/RandomUniform', defined at:
  File "c:\users\19706\appdata\local\programs\python\python36\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\19706\appdata\local\programs\python\python36\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\19706\appdata\local\programs\python\pyth